In [1]:


import torch

print("Torch version:",torch.__version__)

print("Is CUDA enabled?",torch.cuda.is_available())

Torch version: 2.1.0+cpu
Is CUDA enabled? False


Należy dla jedno i dwuwarstwowej sieci neuronowej porównać wyniki i krzywe 
uczenia w zależności od:
• Liczby neuronów w warstwie ukrytej
• Rozmiaru batcha
• Liczby przykładów uczących (należy wykorzystać podzbiory zbioru uczącego o 
różnych rozmiarach zamiast pełnego zbioru uczącego – sprawdzić rozmiary 1% 
danych, 10% danych)
• Zaburzenia danych: dane można zaburzyć dodając do wejściowego batcha 
batch o tych samych wymiarach, wygenerowany jako szum gaussowski o
różnych odchyleniach. Przebadać scenariusze: szum dodany w danych 
testowych vs szum dodany zarówno w testowych, jak i treningowych.

In [21]:
import time
from torchvision import transforms, datasets
import torch
from torch.utils.data import Subset
from torch import nn
from torch import optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.init as init
import random
from sklearn.metrics import precision_score, accuracy_score, f1_score
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)


# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                                 transforms.Normalize((0.5,), (0.5,))])

#how to get train and test data
train_data = datasets.FashionMNIST('path', download=True, train=True, transform=transform)
test_data = datasets.FashionMNIST('path', download=True, train=False, transform=transform)


train_indices = list(range(len(train_data)))
random.shuffle(train_indices)
train_indices_10_percent = train_indices[:len(train_indices)//10]

test_indices = list(range(len(test_data)))
random.shuffle(test_indices)
test_indices_10_percent = test_indices[:len(test_indices)//10]

train_data_10_percent = Subset(train_data, train_indices_10_percent)
test_data_10_percent = Subset(test_data, test_indices_10_percent)


print(len(train_data_10_percent))
print(len(test_data_10_percent))
#Define a dataloader to load data
train_loader = torch.utils.data.DataLoader(train_data_10_percent, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data_10_percent, batch_size=64, shuffle=True)

train_loader_big_batch = torch.utils.data.DataLoader(train_data_10_percent, batch_size=16, shuffle=True)
test_loader_big_batch = torch.utils.data.DataLoader(test_data_10_percent, batch_size=16, shuffle=True)

print(len(train_loader))
print(len(test_loader))    
def train(model, criterion, optimizer, data_loader, test_loader, epochs):
    strart_timestamp = time.time()
    training_loss = []
    test_loss = []
    for epoch in range(epochs):
        for images, labels in data_loader:
            # Flatten images
            images = images.view(images.shape[0], -1)
            # Zero out the gradients
            optimizer.zero_grad()
            # Forward pass, get our logits
            logits = model(images)
            # Calculate the loss with the logits and the labels
            loss = criterion(logits, labels)
            # Backward pass
            loss.backward()
            # Update the weights
            optimizer.step()

        training_loss.append(loss.item())
        if (epoch+1) % 10 == 0:
            print(f"Epoch {epoch+1}/{epochs} - Loss: {loss.item()}")
        with torch.no_grad():
            for images, labels in test_loader:
                # Flatten images
                images = images.view(images.shape[0], -1)
                # Forward pass, get our logits
                logits = model(images)
                # Calculate the loss with the logits and the labels
                loss = criterion(logits, labels)
            test_loss.append(loss.item())

    print(f"\nTraining Time (in minutes) = {(time.time()-strart_timestamp)/60:.2f}")
    print(training_loss, test_loss)
    return training_loss, test_loss

def plot_loss(losses, title):
    train, test = losses
    plt.plot(train)
    plt.plot(test)
    plt.legend(['Train', 'Test'])
    plt.title(title)
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.ylim(0, 3)
    plt.show()

# Define the network architecture
oneLayerModel = nn.Sequential(
                    nn.Linear(784, 128),
                    nn.LeakyReLU(),
                    nn.Linear(128, 10),
                    nn.LogSoftmax(dim = 1))

oneLayerModel_big_batch = nn.Sequential(
                    nn.Linear(784, 128),
                    nn.LeakyReLU(),
                    nn.Linear(128, 10),
                    nn.LogSoftmax(dim = 1))

oneLayerModel_less_neurons = nn.Sequential(
    nn.Linear(784, 32),
                      nn.LeakyReLU(),
                      nn.Linear(32, 10),
                      nn.LogSoftmax(dim = 1))



def init_xavier(m):
    if isinstance(m, nn.Linear):
        init.xavier_normal_(m.weight)
        init.constant_(m.bias, 0)

oneLayerModel.apply(init_xavier)
oneLayerModel_less_neurons.apply(init_xavier)

# Define the loss
criterion = nn.NLLLoss();

# Define the optimizer
oneLayerModel_optimizer = optim.Adam(oneLayerModel.parameters(), lr=0.0002)
oneLayerModel_big_batch_optimizer = optim.Adam(oneLayerModel_big_batch.parameters(), lr=0.0002)
oneLayerModel_less_neurons_optimizer = optim.Adam(oneLayerModel_less_neurons.parameters(), lr=0.0002)
epochs = 100

def evaluate_model(model, test_loader):
    model.eval()
    total = 0
    correct = 0
    all_labels = []
    all_predictions = []

    with torch.no_grad():
        for images, labels in test_loader:
            images = images.view(images.shape[0], -1)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            all_labels.extend(labels.numpy())
            all_predictions.extend(predicted.numpy())

    accuracy = correct / total
    precision = precision_score(all_labels, all_predictions, average='weighted')
    f1 = f1_score(all_labels, all_predictions, average='weighted')

    print(f'Accuracy: {accuracy:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'F1 Score: {f1:.4f}')


oneLayerModel_losses = train(
    oneLayerModel, 
    criterion, 
    oneLayerModel_optimizer, 
    train_loader, test_loader, 
    epochs=epochs)
plot_loss(oneLayerModel_losses, 'Losses')
evaluate_model(oneLayerModel, test_loader)

oneLayerModel_less_neurons_losses = train(
    oneLayerModel_less_neurons, 
    criterion, oneLayerModel_less_neurons_optimizer, 
    train_loader, 
    test_loader, 
    epochs=epochs)
plot_loss(oneLayerModel_less_neurons_losses, 'Losses')
evaluate_model(oneLayerModel_less_neurons, test_loader)


oneLayerModel_losses_big_batch = train(
    oneLayerModel_big_batch, 
    criterion, 
    oneLayerModel_big_batch_optimizer, 
    train_loader_big_batch, 
    test_loader_big_batch, 
    epochs=epochs)
plot_loss(oneLayerModel_losses_big_batch, 'Losses')
evaluate_model(oneLayerModel_big_batch, test_loader_big_batch)



6000
1000
94
16
Epoch 10/100 - Loss: 0.5257911086082458
